
## Example using HF trainer


In [ ]:

import json
import codecs

from transformers import pipeline, set_seed
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForCausalLM, IntervalStrategy, AutoModel, AutoConfig, PreTrainedModel
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Model, PreTrainedModel
from transformers import AutoModelForCausalLM, GPT2PreTrainedModel, GPT2Model
from transformers.modeling_outputs import ModelOutput


import os
import pandas as pd

import torch
from datasets import load_dataset
from torch.utils.data import Dataset, random_split
from torch import nn
from torch.nn import Identity
import torch.nn.functional as F


import json
import deepspeed
from dataclasses import dataclass
from typing import Optional, Tuple
from typing import List

from tqdm import tqdm



In [ ]:

model_checkpoint = "roneneldan/TinyStories-33M"


In [ ]:

ds = load_dataset('MohamedRashad/characters_backstories')["train"]


In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:

def tokenize_function(example):
    merged = example["text"] + " " + example["target"]
    batch = tokenizer(merged, padding='max_length', truncation=True, max_length=128)
    batch["labels"] = batch["input_ids"].copy()
    return batch


In [ ]:

tokenized_dataset = ds.map(tokenize_function, remove_columns=["text", "target"])

model = AutoModelForCausalLM.from_pretrained(model_checkpoint);



In [ ]:

training_args = TrainingArguments(
    num_train_epochs=1,
    output_dir=".",
    no_cuda=True
    # use_mps_device=True,
)


In [ ]:


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)



In [ ]:

print(trainer.accelerator.device)


In [ ]:

# Let's train!
trainer.train()
